In [ ]:
import torch
import torch.nn as nn
import quartz
import matplotlib.pyplot as plt

In [ ]:
t_max = 16

class Net(nn.Module):
    def __init__(self, t_max):
        super().__init__()
        self.t_max = t_max

        # self.weight1 = nn.Linear(1, 1, bias=False)
        self.weight1 = nn.Conv2d(1, 2, 2, bias=False)
        self.layer1 = quartz.IF(t_max=t_max, index=0, record_v_mem=True, rectification=True)

    def forward(self, data: torch.Tensor):
        data = self.weight1(data)
        return self.layer1(data)

    def __len__(self):
        return 1

    def reset_states(self):
        self.layer1.reset_states()

net = Net(t_max=t_max)

In [ ]:
static_data = torch.ones(1, 1, 2, 2)
input_data = quartz.utils.encode_inputs(static_data, t_max=t_max, n_layers=len(net))
n_time_steps = input_data.shape[1]

In [ ]:
net.reset_states()
output = net(input_data)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(6, 4))
ax1.plot(net.layer1.v_mem_recorded.flatten().detach().numpy(), label='v_mem')
ax1.plot(range(n_time_steps), [t_max]*n_time_steps, label='spike threshold')
ax1.plot(output.flatten().detach().numpy(), label='output')

In [ ]:
output[0, -t_max:].nonzero()

In [ ]:
output